# accessing `doit` tasks in posts

`doit` is a powerful developer tool for defining system tasks.
each `doit` task has inputs and outputs that combine to built a computational graph that can be traversed in a smart way. `doit` can be used in incredibly complex build scenarios like jupyterlite or for one off level project tasks

## the `mkdocs` task

we are building the invocation for `mkdocs` for this site.
in this document we:
* define functions to find posts
* define a `doit` task to execute `mkdocs`

find posts and build the mkdocs site

In [7]:
    def task_mkdocs():
        yield dict(name="toc", actions=[
            (set_nav, (pathlib.Path("tonyfast"), "*.ipynb", "mkdocs.yml"))
        ], targets=["mkdocs.yml"], uptodate=[False])
        yield dict(
            name="build",
            actions=["mkdocs build"],
            file_dep=["mkdocs.yml"],
            targets=["site/index.html"]
        )

In [1]:
    import tonyfast, pathlib, yaml, re
    WHERE = pathlib.Path(tonyfast.__file__).parent

`title` is a regex matching `YYYY-MM-DD-TITLE` which is what we consider a valid post;
it must have a date and title.

In [2]:
    title = re.compile("(?P<y>[0-9]{4})-(?P<m>[0-9]{1,2})-(?P<d>[0-9]{1,2})-(?P<t>.+)")

find all the potential notebook posts

In [3]:
    def get_posts_from_dir(dir, glob="*.ipynb"):
        for file in dir.iterdir():
            if file.is_dir():
                if "checkpoints" not in file.name: yield from get_posts_from_dir(file, glob)
            elif file.suffix in {".ipynb"}:
                if (m := title.match(str(file.stem))): yield file, m

    def get_posts(dir, glob="*.ipynb"):
        return sorted((
            (x.relative_to(WHERE), y) for x, y in get_posts_from_dir(dir, glob)
        ), reverse=True, key=lambda x: x[1].group(*"ymd"))

format the ordered entries

In [4]:
    def get_toc(dir, glob="*.ipynb"):
        for x, y in list(get_posts(WHERE, glob)):
            yield " "* 4 + F"- {x}" 

we don't use `yaml` cause our `mkdocs` uses `yaml`  tags.
instead we put the nav at the end of the document
then replace the default with the updated version.

In [5]:
    def get_nav(dir, glob="*.ipynb"):
        return "".join(re.split(
            "(notes\S*:\S*)",
            (WHERE.parent / "mkdocs.yml").read_text(),
            1
        )[:2] )+ "\n" + "\n".join(get_toc(dir, glob))

    def set_nav(dir, glob="*.ipynb", target=WHERE.parent / "mkdocs.yml"):
        pathlib.Path(target).write_text(get_nav(pathlib.Path(dir), glob))

### invocation

the task is exposed in the `tonyfast` module

    python -m tonyfast tasks mkdocs
    
this command is invoked with `hatch` in a virtual environment using:

    hatch run docs:build